In [15]:
#### 双向最大匹配分词方法实现
class BiMM():
    def __init__(self, user_dict):
        self.user_dict = user_dict
    def cut(self, sentence):
        """
        正向最大匹配（FMM）
        :param user_dict: 词典
        :param sentence: 句子
        """
        # 词典中最长词长度
        max_len = max([len(item) for item in self.user_dict])
        start = 0
        f_single_word_num = 0 #单字词数量计数
        f_segs = [] #存储分词结果
        while start != len(sentence):
            index = start+max_len
            if index>len(sentence):
                index = len(sentence)
            for i in range(max_len):
                if (sentence[start:index] in self.user_dict) or (len(sentence[start:index])==1):
                    f_segs.append(sentence[start:index])
                    if index-start==1:
                        f_single_word_num+=1
                    #print(sentence[start:index], end='/')
                    start = index
                    break
                index += -1
        """
        反向最大匹配（BMM）
        :param user_dict:词典
        :param sentence:句子
        """
        result = [] #暂存分词结果
        b_segs = [] #存储分词结果
        b_single_word_num = 0 #单字词数量计数
        start = len(sentence)
        while start != 0:
            index = start - max_len
            if index < 0:
                index = 0
            for i in range(max_len):
                #print(sentence[index:start])
                if (sentence[index:start] in self.user_dict) or (len(sentence[index:start])==1):
                    result.append(sentence[index:start])
                    if start-index == 1:
                        b_single_word_num+=1
                    start = index
                    break
                index += 1
        b_segs = result[::-1]
    
        """
        双向最大匹配
        """
        if len(f_segs)<len(b_segs):
            return f_segs
        elif len(f_segs)>len(b_segs):
            return b_segs
        else:
            if f_single_word_num>b_single_word_num:
                return b_segs
            else:
                return f_segs

In [16]:
#测试双向最大匹配分词方法
user_dict = ['我们','在', '在野', '生动', '野生', '动物园', '野生动物园', '物','园','玩']
sentence = '我们在野生动物园玩'
bimm = BiMM(user_dict)
print(bimm.cut(sentence))

['我们', '在', '野生动物园', '玩']


In [17]:
#从人民日报分词语料中读取分词数据，并构建词典
#将人民日报分词语料还原成原文
ribao_dict = [] #语料中出现的所有词
ribao_dict_sorted = [] #过滤掉低频词后的词典
yuanwen = [] #存储语料原文
punctuation = []  #存储标点符号
yuanwen_segged = []
c = 0
with open("199801.txt","r",encoding="utf-8") as fr, open("199801_source.txt","w",encoding="utf-8") as fw:
    for line in fr:
        #if c > 100:
        #    break
        ls = line.strip().split('  ')
        line_segs = []
        for i in range(1,len(ls)):
            #print(ls[i])
            #if '/' in ls[i]:
            end_idx = ls[i].index('/')
            #line_segs.append(ls[i][:end_idx])
            ribao_dict.append(ls[i][:end_idx])
        
            if ls[i].endswith('/w'):
                punctuation.append(ls[i][:end_idx])
        yuanwen_segged.append(line_segs)
        fw.write(''.join(line_segs)+'\n')

        yuanwen.append(''.join(line_segs))
        #c+=1
word_count_dict = {}
for word in ribao_dict:
    if word not in word_count_dict:
        word_count_dict[word]=1
    else:
        word_count_dict[word]+=1
sorted_word_count = sorted(word_count_dict.items(), key = lambda item:item[1], reverse=True)

#保存出现频次大于2的词，存入词典
for k,v in sorted_word_count:
    if v>2:
        ribao_dict_sorted.append(k)
ribao_dict_sorted = set(ribao_dict_sorted)
punctuation = set(punctuation)
ribao_dict_set = set(ribao_dict)
print("从语料中提取的词典规模：\n",len(ribao_dict_set),len(punctuation), len(ribao_dict_sorted))

从语料中提取的词典规模：
 56482 47 22111


In [18]:
#使用新词典测试双向最大匹配分词方法
bimm = BiMM(ribao_dict_sorted)
bimm.cut("迈向新世纪")

['迈向', '新', '世纪']

In [22]:
import time
from tqdm import tqdm #进度条线上模块

time_start = time.time() #计时
source_text = []
c = 0
with open("199801_source.txt","r",encoding="utf-8") as fr:
    for line in fr:
        c+=1
        if c>2000:
            break
        source_text.append(line.strip())

bimm = BiMM(ribao_dict_sorted)
all_Bi_segs = [] #存储整个语料的分词结果
with open("199801_source_segs.txt","w",encoding="utf-8") as fw:
    for t in tqdm(range(len(source_text)),desc = 'segging'):
        text = source_text[t]  
        start_idx = 0
        Bi_segs = []
        flag = True
        #start_idx = 0
        for idx in range(len(text)):
            if text[idx] in punctuation: #有标点符号的，先根据标点符号节分句子
                Bi_segs.extend(bimm.cut(text[start_idx:idx+1]))
                start_idx = idx+1
                flag = False
        if flag:
            Bi_segs.extend(bimm.cut(text))
        else:
            Bi_segs.extend(bimm.cut(text[start_idx:idx+1]))
        all_Bi_segs.append(Bi_segs)
        fw.write('  '.join(Bi_segs)+'\n')
time_end = time.time()
print('共花费',time_end-time_start,"秒")

segging: 100%|██████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:06<00:00, 29.90it/s]

共花费 66.9113290309906 秒


In [3]:
#计算分词准确率与召回率
#words_num = 0
def countWordNum(xis, yis):
    line_text = ''.join(xis)
    #print(line_text)
    xis_idx_list = []
    yis_idx_list = []
    idx = 0
    for xi in xis:
        eidx = line_text.find(xi,idx)
        xis_idx_list.append(eidx)
        idx=eidx+1
    idx = 0
    for yi in yis:
        eidx = line_text.find(yi,idx)
        yis_idx_list.append(eidx)
        idx=eidx+1
    #print(xis_idx_list)
    #print(yis_idx_list)
    correct_count = 0
    gold_word_count = len(xis_idx_list)-1
    cut_word_count = len(yis_idx_list)-1
    i = 1
    j = 1
    flag = True
    
    while(i<len(xis_idx_list) and j<len(yis_idx_list)):
        if xis_idx_list[i]== yis_idx_list[j]:
            if flag:
                correct_count+=1
            flag=True
            i+=1
            j+=1
        elif xis_idx_list[i] < yis_idx_list[j]:
            i+=1
            flag=False
        else:
            j+=1
            flag=False
    
    #print(xis_idx_list)
    #print(yis_idx_list)
    return correct_count, gold_word_count, cut_word_count  

In [4]:
with open("199801_top_2000.txt","r",encoding="utf-8") as fr1, open("199801_source_top2000_segs.txt","r",encoding="utf-8") as fr2:
    total_correct_num = 0
    total_gold_num = 0
    total_cut_num = 0
    c=0
    while True:
        #if c>1:
        #    break
        if c%1000==0:
            print(c)
        try:
            c+=1
            x = next(fr1)
            y = next(fr2)
            #print("打印分词文本：\n")
            #print(x)
            #print(y)
            xis = []
            yis = []
            for xi in x.strip().split("  ")[1:]:
                #print(xi)
                e_idx = xi.index('/')
                xis.append(xi[:e_idx])
            for yi in y.strip().split("  "):
                #print(yi)
                if '/' in yi:
                    e_idx = yi.index('/')
                    yis.append(yi[:e_idx])
                else:
                    yis.append(yi)
            #xis = [xi[:xi.index('/')] for xi in x.split("  ")]
            #yis = [yi[:yi.index('/')] for yi in y.split("  ")]
            #print(xis)
            #print(yis)
            correct_num, gold_num, cut_num = countWordNum(xis, yis)
            total_correct_num += correct_num
            total_gold_num += gold_num
            total_cut_num += cut_num
            #TT+=TTi
        except StopIteration:
            break
    accuracy = total_correct_num/total_cut_num
    recall = total_correct_num/total_gold_num
    f1 = 2*accuracy*recall/(accuracy+recall)
    print("Accuracy: %.4f\n Recall: %.4f\n F1: %.4f" % (accuracy, recall, f1))
        
        

0
1000
2000
Accuracy: 0.8869
 Recall: 0.9398
 F1: 0.9126


#### 最大概率法分词

In [25]:
# -*- coding: utf-8 -*-
import sys
import os.path
import math
#import nltk
#from nltk.corpus import PlaintextCorpusReader
 
path1 = './icwb2-data-master/gold/pku_training_words.utf8'
path2 = './icwb2-data-master/training/pku_training.utf8'
path3 = './icwb2-data-master/testing/pku_test.utf8'
 
 
with open(path1, 'rt', encoding='utf-8') as f:
    training_words = [w.strip() for w in f.readlines()]
 
#training = PlaintextCorpusReader( *os.path.split(path2) )
#print(training)
#training_words += list(training.words())

with open(path2,'rt',encoding='utf-8') as f:
    for line in f:
        training_words += line.split('  ')
N = len(training_words)
V = len( set(training_words) )
#fdist = nltk.FreqDist(training_words)
fdist = {}
for word in training_words:
    if word not in fdist:
        fdist[word]=1
    else:
        fdist[word]+=1
fdist = dict([(w, math.log((c+1.0)/(N+V))) for w, c in fdist.items()])
defprob = math.log(1.0/(N+V))
 
with open(path3, 'rt', encoding='utf8') as f:
    test = f.readlines()

#DAG, Directed Acyclic Graph, 有向无环图
def get_DAG(sentence):
    DAG = {}
    T = len(sentence)
    for x in range(T):
        ys = []
        for y in range(x+1, T+1):
            if sentence[x:y] in fdist:
                ys.append(y)
        if not ys:
            ys.append(x+1)
        DAG[x] = ys
    return DAG

#深度优先遍历获取最大权重的路径
def dfs(DAG, sentence):
    segments = []
    T = len(sentence)
    def _dfs(words, x):
        for y in DAG[x]:
            if y < T:
                new = words.copy()
                new.append(sentence[x:y])
                _dfs(new, y)
            else:
                new = words.copy()
                new.append(sentence[x:y])
                segments.append( new )
    _dfs([], 0)
    bestseg = max([(sum(fdist.get(w, defprob) for w in seg), seg) for seg in segments])
    return bestseg[1]
 
def dp(DAG, sentence):
    T = len(sentence)
    prob = {T:(0.0,)}
    for x in range(T-1, -1, -1):
        prob[x] = max([(fdist.get(sentence[x:y], defprob) + prob[y][0], y) for y in DAG[x]])
    x = 0
    bestseg = []
    while x < T:
        y = prob[x][1]
        bestseg.append( sentence[x:y] )
        x = y
    return bestseg
 
for sent in test[:5]:
    DAG = get_DAG(sent)
    print("有向无环图：",DAG)
    #seg1 = dfs(DAG, sent)
    seg2 = dp(DAG, sent)
    #print('  '.join(seg1), sep='', end='')
    print('  '.join(seg2), sep='', end='')
    #break

有向无环图： {0: [1, 2], 1: [2, 3], 2: [3, 4], 3: [4], 4: [5, 6], 5: [6], 6: [7], 7: [8, 10], 8: [9, 10], 9: [10], 10: [11, 12], 11: [12], 12: [13], 13: [14], 14: [15], 15: [16], 16: [17], 17: [18, 19], 18: [19], 19: [20, 21], 20: [21], 21: [22]}
共同  创造  美好  的  新世纪  ——  二  ○  ○  一  年  新年  贺词  
有向无环图： {0: [1], 1: [2, 6], 2: [3], 3: [4], 4: [5], 5: [6], 6: [7, 8, 9], 7: [8, 9], 8: [9], 9: [10, 11, 12, 13], 10: [11, 12, 13], 11: [12, 13], 12: [13], 13: [14], 14: [15], 15: [16], 16: [17, 18], 17: [18], 18: [19], 19: [20], 20: [21], 21: [22]}
（  二○○○年  十二月  三十一日  ）  （  附  图片  1  张  ）  
有向无环图： {0: [1, 2], 1: [2], 2: [3], 3: [4], 4: [5, 6], 5: [6], 6: [7], 7: [8], 8: [9, 10], 9: [10], 10: [11], 11: [12], 12: [14], 13: [14], 14: [15], 15: [16], 16: [17]}
女士  们  ，  先生  们  ，  同志  们  ，  朋友  们  ：  
有向无环图： {0: [1], 1: [2], 2: [3], 3: [4], 4: [5], 5: [6, 7], 6: [7], 7: [8, 9], 8: [9], 9: [10, 11], 10: [11], 11: [12, 13], 12: [13], 13: [14], 14: [15, 16], 15: [16], 16: [17, 18], 17: [18, 19], 18: [19, 20],

#### HMM分词与序列标注

In [26]:
# -*- coding: utf-8 -*-
import sys, re, math
 
#sys.argv.append('./gold/pku_training_words.utf8')
#sys.argv.append('./training/pku_training.utf8')
#sys.argv.append('./testing/pku_test.utf8')

training_words_filename = './icwb2-data-master/gold/pku_training_words.utf8'
training_filename = './icwb2-data-master/training/pku_training.utf8'
test_filename = './icwb2-data-master/testing/pku_test.utf8'

 
with open(training_words_filename, 'rt', encoding='utf8') as f:
    training_words = f.readlines()
 
with open(training_filename, 'rt', encoding='utf8') as f:
    training = f.readlines()
 
with open(test_filename, 'rt', encoding='utf8') as f:
    test = f.readlines()
 
# word tag by char
A, B, P = {}, {}, {}
for line in training:
    #print( line )
    prev_a = None
    for w, word in enumerate(re.split(r'\s{2}', line)):
        I = len(word)
        for i, c in enumerate(word):
            if I == 1:
                a = 'S'
            else:
                if i == 0:
                    a = 'B'
                elif i == I-1:
                    a = 'E'
                else:
                    a = 'M'
            # print(w, i, c, a)
            if prev_a is None: # calculate Initial state Number
                if a not in P: 
                    P[a] = 0
                P[a] += 1
            else: # calculate State transition Number
                if prev_a not in A: 
                    A[prev_a] = {}
                if a not in A[prev_a]: 
                    A[prev_a][a] = 0
                A[prev_a][a] += 1
            prev_a = a
            # calculate Observation Number
            if a not in B: 
                B[a] = {}
            if c not in B[a]: 
                B[a][c] = 0
            B[a][c] += 1
 
# calculate probability
for k, v in A.items():
    total = sum(v.values())
    A[k] = dict([(x, math.log(y / total)) for x, y in v.items()])

for k, v in B.items():
    # plus 1 smooth
    total = sum(v.values())
    V = len(v.values())
    B[k] = dict([(x, math.log((y+1.0) / (total+V))) for x, y in v.items()])
    # plus 1 smooth
    B[k]['<UNK>'] = math.log(1.0 / (total+V))

minlog = math.log( sys.float_info.min )
total = sum(P.values())
for k, v in P.items():
    P[k] = math.log( v / total )
 
def viterbi(observation):
    state = ['B','M','E','S']
    T = len(observation)
    delta = [None] * (T + 1)
    psi = [None] * (T + 1)
    delta[0] = dict([(i, P.get(i, minlog) + B[i].get(observation[0], B[i]['<UNK>'])) for i in state])
    psi[0] = dict([(i, None) for i in state])
    for t in range(1, len(observation)):
        Ot = observation[t]
        delta[t] = dict([(j, max([delta[t-1][i] + A[i].get(j, minlog) + B[j].get(Ot, B[j]['<UNK>']) for i in state])) for j in state])
        psi[t] = dict([(j, max([(delta[t-1][i] + A[i].get(j, minlog), i) for i in state])[1]) for j in state ])
    delta[T] = max( [ delta[T-1][i] for i in state ] )
    psi[T] = max([(delta[T-1][i], i) for i in state ] )[1]
    q = [None] * (T+1)
    q[T] = psi[T]
    for t in range(T-1, -1, -1):
        q[t] = psi[t][q[t+1]]
    return q[1:]
 
for sent in test[:5]:
    sequence = viterbi( list(sent) )
    segment = []
    for char, tag in zip(sent, sequence):
        if tag == 'B':
            segment.append(char)
        elif tag == 'M':
            segment[-1] += char
        elif tag == 'E':
            segment[-1] += char
        elif tag == 'S':
            segment.append(char)
        else:
            raise Exception()
    print('  '.join(segment), sep='', end='')
    #break

共同  创造  美好  的  新  世纪  ——二○○一年  新年  贺词  
（  二○○○年  十二月  三十  一日  ）  （  附  图片  1张  ）  
女士  们  ，  先  生们  ，  同志  们  ，  朋友  们  ：  
2001年  新年  钟声  即  将  敲响  。  人类  社会  前进  的  航船  就  要驶入21世纪  的  新  航程  。  中国  人民  进入  了  向  现代化  建设  第三步  战略  目标  迈进  的  新  征程  。  
在  这个  激动  人心  的  时刻  ，  我  很  高兴  通过  中国  国际  广播  电台  、  中央  人民  广播  电台  和  中央  电视  台  ，  向  全国  各族  人民  ，  向  香港  特别  行政区  同胞  、  澳门  特别  行政区  同胞  和  台湾  同胞  、  海外  侨胞  ，  向  世界  各国  的  朋友  们  ，  致以  新  世纪  第一  个  新年  的  祝贺  ！  
